
# Projet 2

Projet maison 1 et 2 (semaine 39)

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [19]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
#geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape
Code INSEE,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273..."
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089..."
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190..."
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580..."
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854..."


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [3]:
# on ajoute la colonne "CP Ville"
geo["CP Ville"] = geo["Code Postal"] + " " + geo["Commune"]
geo.head(1)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,CP Ville
Code INSEE,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",01400 L'ABERGEMENT-CLEMENCIAT


In [4]:
# extraire Latitude et Longitude de "geo_point_2d"
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo.head(1)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,CP Ville,Latitude,Longitude
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",01400 L'ABERGEMENT-CLEMENCIAT,46.153426,4.926114


In [5]:
# fonction recherche de ville
def ville(lat, lon):
    s =haversine_np(geo['Latitude'], geo['Longitude'], lat, lon)
    print("\n-----------------------------------------------------------------")
    print(f'La plus proche commune est {geo.loc[s.idxmin()]["Commune"].title()} ({geo.loc[s.idxmin()]["Code Postal"]}) située à une distance de {s.min():.3f} km')
    print("-----------------------------------------------------------------\n")

# fonction calcul de distance
def haversine_np(lat1, lon1, lat2, lon2):
    """
    Calcule la distance courbe entre 2 points
    
    Retourne la distance en km
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    diff_lat = lat2 - lat1
    diff_lon = lon2 - lon1

    a = np.sin(diff_lat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(diff_lon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [6]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [9]:
# on génère une coordonnée
def random_coordinates():
    # on applique la fonction à une coordonnée tirée au hasard
    # init du random
    np.random.seed(0)
    a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
    lat = (b - a) * np.random.random() + a
    a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
    lon = (b - a) * np.random.random() + a    
    return lat, lon

lat, lon = random_coordinates()

# on applique la fonction à une coordonnée tirée au hasard
print('Distance commune la plus proche (lat/lon)')
ville(lat, lon)

Distance commune la plus proche (lat/lon)

-----------------------------------------------------------------
La plus proche commune est Bosjean (71330) située à une distance de 0.436 km
-----------------------------------------------------------------



In [10]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))


-----------------------------------------------------------------
La plus proche commune est Palaiseau (91120) située à une distance de 1.248 km
-----------------------------------------------------------------



**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [9]:
# import des modules usuels
import numpy as np
import pandas as pd

# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()

In [10]:
import json

# recuperer lae champ geo_shape
gs=geo["geo_shape"]
# utilisation d'une série
serie = gs.apply(json.loads)
serie


Code INSEE
01001    {'type': 'Polygon', 'coordinates': [[[4.926273...
01002    {'type': 'Polygon', 'coordinates': [[[5.430089...
01004    {'type': 'Polygon', 'coordinates': [[[5.386190...
01005    {'type': 'Polygon', 'coordinates': [[[4.895580...
01006    {'type': 'Polygon', 'coordinates': [[[5.614854...
                               ...                        
97613    {'type': 'Polygon', 'coordinates': [[[45.10168...
97614    {'type': 'Polygon', 'coordinates': [[[45.15401...
97615    {'type': 'Polygon', 'coordinates': [[[45.29645...
97616    {'type': 'Polygon', 'coordinates': [[[45.13226...
97617    {'type': 'Polygon', 'coordinates': [[[45.15256...
Name: geo_shape, Length: 36742, dtype: object

In [12]:
js = geo["geo_shape"].apply(lambda x: json.loads(x))
geo['type'] = js.apply(lambda x: x['type'])
geo['coordinates'] = js.apply(lambda x: x['coordinates'])
geo.head(2)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,type,coordinates
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",Polygon,"[[[4.92627342692396, 46.1200517234555], [4.921..."
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",Polygon,"[[[5.430089291292173, 45.98275997279625], [5.4..."


In [14]:
# Nombre de valeurs accédées avec la clé "type".

l2 = geo['type'].count()
print(f'Il y {l2} valeurs avec la clé "type"')


Il y 36742 valeurs avec la clé "type"


In [15]:
# Liste des longeurs de chaque liste de coordonnées
longs = geo['coordinates'].apply(lambda x: len(x[0]))
longs

Code INSEE
01001    17
01002    10
01004    16
01005    13
01006    10
         ..
97613    14
97614    12
97615     7
97616    13
97617    15
Name: coordinates, Length: 36742, dtype: int64

In [16]:
maxlong=max(longs)
print(f'La commune la plus complexe géométriquement comporte {max(longs)} côtés')



La commune la plus complexe géométriquement comporte 486 côtés


In [18]:
multipols = geo[geo["type"] == "MultiPolygon"]
maxm = multipols['coordinates'].apply(lambda x: len(x[0][0])).max()

print(f'Le MultiPolygon le plus long et compliqué comporte {maxm} éléments')

Le MultiPolygon le plus long et compliqué comporte 58 éléments
